## Importing

In [ ]:
%pip install rdflib
%pip install meteostat
%pip install geopy

import pandas as pd
import csv
from pathlib import Path
from urllib.parse import quote
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import XSD, RDFS, WGS
from datetime import datetime, timedelta
from meteostat import Point, Hourly
import warnings
import hashlib 
from geopy.distance import geodesic as GD

## Settings

In [2]:
# Define paths
# Input paths
BASE_PATH = Path(r"D:\\OneDrive - Università degli Studi di Padova\\Lezioni\\Magistrale\\Terzo semestre\\Graph DB\\Homework\\flydata")
STATE_PATH = BASE_PATH / "DataCollection" / "CSVData" / "states.csv"
CITY_PATH = BASE_PATH / "DataCollection" / "CSVData" / "cities.csv"
AIRPORT_PATH = BASE_PATH / "DataCollection" / "CSVData" / "airports.csv"
CARRIER_PATH = BASE_PATH / "DataCollection" / "CSVData" / "carriers.csv"
AIRCRAFT_PATH = BASE_PATH / "DataCollection" / "CSVData" / "aircrafts.csv"
MODEL_PATH = BASE_PATH / "DataCollection" / "CSVData" / "model.csv"
FLIGHT_PATH = BASE_PATH / "DataCollection" / "CSVData" / "flight.csv"
MANUFACTURER_PATH = BASE_PATH / "DataCollection" / "CSVData" / "manufacturer.csv"
# Output paths
OUTPUT_AIRPORT_PATH = BASE_PATH / "Serialization" /"ttl"/ "airports.ttl"
OUTPUT_CARRIER_PATH = BASE_PATH / "Serialization" / "ttl" / "carriers.ttl"
OUTPUT_STATE_PATH = BASE_PATH / "Serialization" /"ttl"/ "states.ttl"
OUTPUT_CITY_PATH = BASE_PATH / "Serialization" /"ttl"/ "cities.ttl"
OUTPUT_AIRCRAFT_PATH = BASE_PATH / "Serialization" /"ttl"/ "aircrafts.ttl"
OUTPUT_MODEL_PATH = BASE_PATH / "Serialization" /"ttl"/ "models.ttl"
OUTPUT_WEATHER_PATH = BASE_PATH / "Serialization" /"ttl"/ "weather.ttl"
OUTPUT_FLIGHT_PATH = BASE_PATH / "Serialization" /"ttl"/ "flights.ttl"
OUTPUT_ROUTE_PATH = BASE_PATH / "Serialization" /"ttl"/ "routes.ttl"
OUTPUT_MANUFACTURER_PATH = BASE_PATH / "Serialization" / "ttl" / "manufacturers.ttl"

# Define namespaces
FDO = Namespace("http://www.semanticweb.org/nele/ontologies/2024/10/flydata/")

# Define the time interval for the weather data
start = pd.to_datetime('2024-8-1')
end = pd.to_datetime('2024-8-31')

## State

In [ ]:
g_state = Graph()

g_state.bind("fdo", FDO)
g_state.bind("xsd", XSD)

states = []
try:
    with open(STATE_PATH, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['State'] and row['Abbreviation']:  # Need both fields
                state_dict = {
                    'name': row['State'].strip(),
                    'abbreviation': row['Abbreviation'].strip()
                }
                states.append(state_dict)
except Exception as e:
    print(f"Error reading states file: {str(e)}")
    raise

# Create a mapping of state abbreviations to URIs
for state in states:
    state_id = quote(state['name'].encode('ascii', 'ignore').decode().replace(" ", "_").replace("'", "").replace(",", ""))
    g_state.add((URIRef(str(FDO) + state_id), RDF.type, FDO.State))
    g_state.add((URIRef(str(FDO) + state_id), FDO.name, Literal(state['name'], datatype=XSD.string)))
    g_state.add((URIRef(str(FDO) + state_id), FDO.abbreviation, Literal(state['abbreviation'], datatype=XSD.string)))

g_state.serialize(destination=str(OUTPUT_STATE_PATH), format='turtle')

## Cities

In [ ]:
g_city = Graph()

g_city.bind("fdo", FDO)
g_city.bind("xsd", XSD)
g_city.bind("wgs", WGS)

cities = {}
try:
    # Load the CSV file in memory using pandas
    cities_df = pd.read_csv(CITY_PATH)
    for _, row in cities_df.iterrows():
        # Convert city_ascii to string and skip if it's NaN
        if pd.isna(row['city_ascii']):
            continue
        
        city_id = quote(row['city_ascii'].encode('ascii', 'ignore').decode().replace(" ", "_").replace("'", "").replace(",", "")+str(hash(row['lat']+row['lng'])))
        cities[city_id] = {
            'name': row['city_ascii'],
            'population': str(row['population']) if pd.notna(row['population']) else "0",
            'state_name': str(row['state_name']) if pd.notna(row['state_name']) else "",
            'state_id': str(row['state_id']) if pd.notna(row['state_id']) else "",
            'lat': str(row['lat']) if pd.notna(row['lat']) else "",
            'lng': str(row['lng']) if pd.notna(row['lng']) else ""
        }
                    
except Exception as e:
    print(f"Error reading file: {str(e)}")
    raise

# Add cities to the city graph
for city_id, data in cities.items():
    # Remove special characters and spaces, then URL encode
    city_uri = FDO[city_id]
    
    # Add city triples
    g_city.add((city_uri, RDF.type, FDO.City))
    g_city.add((city_uri, FDO.name, Literal(data['name'], datatype=XSD.string)))
    g_city.add((city_uri, FDO.population, Literal(data['population'], datatype=XSD.nonNegativeInteger)))
    g_city.add((city_uri, FDO.latitude, Literal(data['lat'], datatype=WGS.lat)))
    g_city.add((city_uri, FDO.longitude, Literal(data['lng'], datatype=WGS.long)))

    if data['state_name'] and any(state['name'] == data['state_name'] for state in states):  # Check if state_name exists and is in states
        g_city.add((city_uri, FDO.isLocatedInState, FDO[data['state_name'].encode('ascii', 'ignore').decode().replace(" ", "_").replace("'", "").replace(",", "")]))

g_city.serialize(destination=str(OUTPUT_CITY_PATH), format='turtle')

## Airports

In [ ]:
g_airport = Graph()

g_airport.bind("fdo", FDO)
g_airport.bind("xsd", XSD)
g_airport.bind("wgs", WGS)

# Define the isLocatedInCity property in the airport graph
g_airport.add((FDO.isLocatedInCity, RDF.type, RDF.Property))
g_airport.add((FDO.isLocatedInCity, RDFS.domain, FDO.Airport))
g_airport.add((FDO.isLocatedInCity, RDFS.range, FDO.City))

airports = []
try:
    with open(AIRPORT_PATH, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['IATA']:  # Only include airports with IATA code
                airports.append({
                    'name': row['Name'].strip('"'),
                    'city': row['City'].strip('"'),
                    'iata': row['IATA'].strip('"'),
                    'lat': row['LAT'].strip('"'),
                    'lng': row['LONG'].strip('"')
                })
except Exception as e:
    print(f"Error reading airports file: {str(e)}")
    raise

index = 0
# Add airports and their relationships to the airport graph
for airport in airports:
    # Use IATA code directly in the URI
    airport_uri = URIRef(str(FDO) + airport['iata'])
    
    # Add airport triples
    g_airport.add((airport_uri, RDF.type, FDO.Airport))
    g_airport.add((airport_uri, FDO.name, Literal(airport['name'], datatype=XSD.string)))
    g_airport.add((airport_uri, FDO.latitude, Literal(airport['lat'], datatype=WGS.lat)))
    g_airport.add((airport_uri, FDO.longitude, Literal(airport['lng'], datatype=WGS.long)))
    
    # Find the closest city to the airport
    closest_city_uri = None
    min_distance = float('inf')
    citiesuris = [city_id for city_id, data in cities.items() if data['name'] == airport['city']]
    
    for city_id in citiesuris:
        city_lat = float(cities[city_id]['lat'])
        city_lng = float(cities[city_id]['lng'])
        # Calculate the distance between the city and the airport
        distance = GD((city_lat, city_lng), (float(airport['lat']), float(airport['lng']))).km
        if distance < min_distance:
            min_distance = distance
            closest_city_uri = FDO[city_id] 
    city_uri = closest_city_uri

    if city_uri:
        g_airport.add((airport_uri, FDO.isLocatedInCity, city_uri))
     
# Serialize both graphs to separate TTL files
g_airport.serialize(destination=str(OUTPUT_AIRPORT_PATH), format='turtle')

## Carrier

In [ ]:
g_carrier = Graph()

g_carrier.bind("fdo", FDO)
g_carrier.bind("xsd", XSD)

# populate the carrier graph
carriers = []
try:
    with open(CARRIER_PATH, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['IATA']:  # Only include carriers with IATA code
                carriers.append({
                    'name': row['Name'].strip('"'),
                    'iata': row['IATA'].strip('"'),
                    'callsign': row['Callsign'].strip('"'),
                    'airline_id': row['\ufeffAirlineID'].strip('"')  # Updated to match exact column name with BOM
                })
except Exception as e:
    print(f"Error reading carriers file: {str(e)}")
    raise

for carrier in carriers:
    carrier_uri = FDO[carrier['iata']]
    g_carrier.add((carrier_uri, RDF.type, FDO.Carrier))
    if carrier['callsign']:
        g_carrier.add((carrier_uri, FDO.callSign, Literal(carrier['callsign'], datatype=XSD.string)))
    g_carrier.add((carrier_uri, FDO.name, Literal(carrier['name'], datatype=XSD.string)))

g_carrier.serialize(destination=str(OUTPUT_CARRIER_PATH), format='turtle')

## Manufacturer

In [ ]:
hasher = hashlib.sha1() 

g_manufacturer = Graph()

g_manufacturer.bind("fdo", FDO)
g_manufacturer.bind("xsd", XSD)

manufacturers = []
try:
   with open(MANUFACTURER_PATH, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            hasher.update(row['ManufacturerName'].strip().encode('utf-8'))
            manufacturer_id = 'mf'+hasher.hexdigest()[:16]
            manufacturers.append({
                'manufacturer_id': manufacturer_id,
                'manufacture_code': row['ManufactureCode'].strip(),
                'manufacturer_name': row['ManufacturerName'].strip()
            })
except Exception as e:
    print(f"Error reading aircrafts file: {str(e)}")
    raise

index = 0
for manufacturer in manufacturers:
    if (FDO[manufacturer['manufacturer_id']], RDF.type, FDO.Manufacturer) not in g_manufacturer:
        manufacturer_uri = FDO[manufacturer['manufacturer_id']]
        g_manufacturer.add((manufacturer_uri, RDF.type, FDO.Manufacturer))
        g_manufacturer.add((manufacturer_uri, FDO.name, Literal(manufacturer['manufacturer_name'], datatype=XSD.string)))

    print(f"{int(index/len(manufacturers)*100)} % complete \r", end="")
    index+=1

g_manufacturer.serialize(destination=str(OUTPUT_MANUFACTURER_PATH), format='turtle')

## Model

In [ ]:
g_model = Graph()

g_model.bind("fdo", FDO)
g_model.bind("xsd", XSD)

models = []
try:
    with open(MODEL_PATH, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            models.append({
                'model_code': row['ModelCode'].strip(),
                'model_name': row['ModelName'].strip(),
            })
except Exception as e:
    print(f"Error reading model file: {str(e)}")
    raise
index = 0
for model_data in models:
    model_uri = URIRef(str(FDO) + quote(model_data['model_code']))
    g_model.add((model_uri, RDF.type, FDO.Model))
    g_model.add((model_uri, FDO.name, Literal(model_data['model_name'], datatype=XSD.string)))
    g_model.add((model_uri, FDO.modelCode, Literal(model_data['model_code'], datatype=XSD.string)))

    manufacturer_uri = None
    for manufacturer in manufacturers:
        if manufacturer['manufacture_code'] == model_data['model_code']:
            manufacturer_uri = FDO[manufacturer['manufacturer_id']]
            g_model.add((model_uri, FDO.hasManufacturer, manufacturer_uri))
            break

    print(f"{int(index/len(models)*100)} % complete \r", end="")
    index+=1

print(f"serialization \r")
g_model.serialize(destination=str(OUTPUT_MODEL_PATH), format='turtle')

## Aircraft

In [ ]:
g_aircraft = Graph()

g_aircraft.bind("fdo", FDO)
g_aircraft.bind("xsd", XSD)

aircrafts = []
try:
    with open(AIRCRAFT_PATH, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            aircrafts.append({
                'n_number': row['AircraftID'],
                'model_code': row['ModelCode'].strip(),
                'aircraft_type': row['AircraftType'].strip(),
                'register_city': row['RegisterCity'].strip()
            })
except Exception as e:
    print(f"Error reading aircrafts file: {str(e)}")
    raise

index = 0
for aircraft in aircrafts:
    encoded_n_number = quote(aircraft['n_number'].strip())
    aircraft_uri = FDO[encoded_n_number]
    g_aircraft.add((aircraft_uri, RDF.type, FDO.Aircraft))
    g_aircraft.add((aircraft_uri, FDO.aircraftType, Literal(aircraft['aircraft_type'], datatype=XSD.string)))

    for model in models:
        if aircraft['model_code'] == model['model_code']:
            model_code = aircraft['model_code']
            model_uri = FDO[model_code]
            g_aircraft.add((aircraft_uri, FDO.hasModel, model_uri))
    print(f"{int(index/len(aircrafts)*100)} % complete \r", end="")
    index+=1

print(f"serialization \r")
g_aircraft.serialize(destination=str(OUTPUT_AIRCRAFT_PATH), format='turtle')

## Weather

In [ ]:
#suppression of the meteostat warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

g_weather = Graph()

g_weather.bind("fdo", FDO)
g_weather.bind("xsd", XSD)

# for each airport, get the weather data
index = 0
for airport in airports:
    # Try to get the weather data for each airport
    try:   
        # Get the weather data
        point = Point(float(airport['lat']), float(airport['long']), 0)
        data = Hourly(point, start, end)
        data = data.fetch()
        
        # Create the node to add to the Graph
        for idx, row in data.iterrows():
            # the node has the namespace + the airport iata + timestamp as URI
            airport_uri = URIRef(str(FDO) + airport['iata'] + str(int(row.name.timestamp()/1000)))
            
            # Add airport triples
            g_weather.add((airport_uri, RDF.type, FDO.Weather))
            g_weather.add((airport_uri, FDO['weatherDate'], Literal(int(row.name.timestamp()), datatype=XSD.dateTime)))
            coco_standardized = 0 if pd.isna(row.coco) else int(row.coco)
            g_weather.add((airport_uri, FDO['weatherType'], Literal(coco_standardized, datatype=XSD.nonNegativeInteger)))
            g_weather.add((airport_uri, FDO['hasAirport'], URIRef(FDO[airport['iata']])))
    except Exception as e:
        print(f"Error weather ({airport['iata']}): {str(e)}")

    print(f"{int(index/len(airports)*100)} % complete \r", end="")
    index+=1

print(f"serialization \r")
g_weather.serialize(destination=str(OUTPUT_WEATHER_PATH), format='turtle')

## Flights and routes

In [ ]:
g_flights = Graph()
g_routes = Graph()

g_flights.bind("fdo", FDO)
g_flights.bind("xsd", XSD)
g_routes.bind("fdo", FDO)
g_routes.bind("xsd", XSD)

timefieldsontology = {
        'Scheduled departure time as shown in Official Airline Guide(OAG)': 'ScheduledDepartureOAGTime',
        'Scheduled departure time as shown in CRS(selected by the Carrier)': 'ScheduledDepartureCRSTime',
        'Gate departure time (actual)': 'ActualGateDepartureTime',
        'Scheduled arrival time per OAG': 'ScheduledArrivalTimePerOAG',
        'Scheduled arrival time per CRS': 'ScheduledArrivalTimePerCRS',
        'Gate arrival time (actual)': 'ActualGateArrivalTime',
        'Wheels-off time (actual)': 'ActualWheels-offTime',
        'Wheels-on time (actual)': 'ActualWheels-onTime'
    }
minutesfieldsontology = {
    'Scheduled elapsed time per CRS': 'ScheduledElapsedTimePerCRS',
    'Actual gate-to-gate time': 'ActualGate-to-gateTime',
    'Departure delay time (actual minutes)': 'ActualDepartureDelayTime',
    'Arrival delay time (actual minutes)': 'ActualArrivalDelayTime',
    'Elapsed time difference (actual minutes)': 'ActualElapsedTimeDifference',
    'Minutes late for Delay Code E - Carrier Caused': 'LateE',
    'Minutes late for Delay Code F - Weather': 'LateF',
    'Minutes late for Delay Code G - National Aviation System (NAS)': 'LateG',
    'Minutes late for Delay Code H - Security': 'LateH',
    'Minutes late for Delay Code I - Late Arriving Flight (Initial)': 'LateI'
}

try:
    # Load the CSV file in memory using pandas
    flights_df = pd.read_csv(FLIGHT_PATH, dtype={"Scheduled Operating Carrier Code": "string", "Date of flight operation": "string"})
    # Fill NaN values in "Actual Operating Carrier Flight Number" with 0 and convert to int
    flights_df["Actual Operating Carrier Flight Number"] = flights_df["Actual Operating Carrier Flight Number"].fillna(0).astype(int)

except Exception as e:
    print(f"Error reading file: {str(e)}")
    raise

index = 0
# Add flights to the flights graph
for idx, row in flights_df.iterrows():

    route_id = quote(str(row['Departure airport code']) + str(row['Arrival airport code']))
    route_uri = URIRef(str(FDO) + route_id)

    # Add route triples
    # check if the route already has the carrier
    if (route_uri, FDO['hasCarrier'], URIRef(FDO[row['Marketing Carrier code']])) not in g_routes:
        # check if the route already exists
        if (route_uri, RDF.type, FDO.Route) not in g_routes:
            g_routes.add((route_uri, RDF.type, FDO.Route))
            g_routes.add((route_uri, FDO['hasDepartureAirport'], URIRef(FDO[row['Departure airport code']])))
            g_routes.add((route_uri, FDO['hasArrivalAirport'], URIRef(FDO[row['Arrival airport code']])))
        # add the carrier to the route
        g_routes.add((route_uri, FDO['hasCarrier'], URIRef(FDO[row['Marketing Carrier code']])))

    flight_date = datetime.strptime(row['Date of flight operation'], "%m/%d/%Y")
    flight_id = quote(str(row['Actual Operating Carrier Code']) + str(row['Actual Operating Carrier Flight Number']) + flight_date.strftime("%Y%m%d"))
    flight_uri = URIRef(str(FDO) + flight_id)

    start_time = row['Scheduled departure time as shown in Official Airline Guide(OAG)']
    for time in timefieldsontology.keys():
        if pd.isna(row[time]) or row[time] == 0:
            row[time] = None
            continue
        # convert the time to a datetime object
        m = int(str(int(row[time]))[-2:])
        h = int(str(int(row[time]))[:2]) if len(str(row[time])) == 4 else 0
        # check if the time is in the next day
        d = 1 if row[time] < start_time-200 else 0
        row[time] = flight_date + timedelta(days=d, hours=h, minutes=m)

    # Add flights triples
    g_flights.add((flight_uri, RDF.type, FDO.Flight))
    g_flights.add((flight_uri, FDO['hasAircraft'], URIRef(FDO[str(row['Aircraft tail number'])])))
    g_flights.add((flight_uri, FDO['hasRoute'], route_uri))
    g_flights.add((flight_uri, FDO['isOperatedBy'], URIRef(FDO[str(row['Actual Operating Carrier Code'])])))
    g_flights.add((flight_uri, FDO['isSoldBy'], URIRef(FDO[str(row['Marketing Carrier Code'])])))
    if not pd.isna(row['Cancellation code']):
        g_flights.add((flight_uri, FDO['CancellationCode'], Literal(row['Cancellation code'], datatype=XSD.string)))
    g_flights.add((flight_uri, FDO['flightDate'], Literal(flight_date, datatype=XSD.dateTime)))

    # time fields
    for original, ontology in timefieldsontology.items():
        if not pd.isna(row[original]):
            g_flights.add((flight_uri, FDO[ontology], Literal(row[original], datatype=XSD.dateTime)))
    # minutes fields
    for original, ontology in minutesfieldsontology.items():
        if not pd.isna(row[original]):
            g_flights.add((flight_uri, FDO[ontology], Literal(int(row[original]), datatype=XSD.integer)))

    index+=1
    print(f"{int(index/len(flights_df)*100)} % complete \r", end="\r")

print(f"serialization\r")
g_flights.serialize(destination=str(OUTPUT_FLIGHT_PATH), format='turtle')
g_routes.serialize(destination=str(OUTPUT_ROUTE_PATH), format='turtle')